<a href="https://colab.research.google.com/github/ryanc0sta/EDA_California/blob/main/EDA_California.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Análise Exploratória de Dados completa com representações gráficas do Dataset [California Housing Prices](https://raw.githubusercontent.com/TailUFPB/processo2fase/refs/heads/master/2025/data/california.csv).

In [ ]:
### IMPORTAÇÕES E CONFIGURAÇÕES


### Obs: utilizei ia principalmente na organização e no visual da criação de gráficos, parte que
### infelizmente ainda não dominei por completo.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 12

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/TailUFPB/processo2fase/refs/heads/master/2025/data/california.csv')
df

### VISÃO GERAL DO DATASET

def visao_geral(df):
    print("\n" + "-"*80)
    print("INFORMAÇÕES GERAIS DO DATASET")
    print("-"*80)

    ### Informações básicas
    print(f"\nDimensões: {df.shape[0]} linhas × {df.shape[1]} colunas")
    print(f"Memória utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print("\nTipos de Dados:")
    print(df.dtypes.value_counts())

    print("\nInformações:")
    df.info()

teste = visao_geral(df)
teste

In [ ]:
### CÁLCULO ESTATÍSTICO DETALHADO (visualização com dataframe)

def calcular_estatisticas_detalhadas(df):
    resumo = {}
    total_registros = len(df)

    for col in df.columns:
        col_data = df[col]
        is_numeric = pd.api.types.is_numeric_dtype(col_data)
        is_categorical = pd.api.types.is_categorical_dtype(col_data)

        ### Aqui faço o pré-cálculo de valores que se repetirão para otimizar o desempenho
        valores_faltantes = col_data.isnull().sum()
        perc_faltantes = (valores_faltantes / total_registros * 100)

        ### Inicializar stats base (comuns a todos os tipos)
        stats = {
            'Tipo': str(col_data.dtype),
            'Total Registros': total_registros,
            'Valores Únicos': col_data.nunique(),
            'Valores Faltantes': valores_faltantes,
            '% Faltantes': f"{perc_faltantes:.2f}%",
        }

        ### Processando os valores numéricos
        if is_numeric:
            zeros = (col_data == 0).sum()
            media = col_data.mean()
            mediana = col_data.median()
            desvio = col_data.std()
            minimo = col_data.min()
            maximo = col_data.max()

            ### Quartis
            q1 = col_data.quantile(0.25)
            q3 = col_data.quantile(0.75)
            iqr = q3 - q1

            ### Outliers
            limite_inf = q1 - 1.5 * iqr
            limite_sup = q3 + 1.5 * iqr
            outliers = ((col_data < limite_inf) | (col_data > limite_sup)).sum()

            ### Moda
            moda_values = col_data.mode()
            moda = moda_values[0] if len(moda_values) > 0 else '-'

            ### Assimetria e Curtose
            assimetria = col_data.skew()
            curtose = col_data.kurtosis()

            ### CV
            cv = (desvio / media * 100) if media != 0 else None

            ### Adicionando ao dicionário
            stats.update({
                'Zeros': zeros,
                '% Zeros': f"{(zeros / total_registros * 100):.2f}%",
                'Média': f"{media:.2f}",
                'Mediana': f"{mediana:.2f}",
                'Moda': moda,
                'Desvio Padrão': f"{desvio:.2f}",
                'CV (%)': f"{cv:.2f}%" if cv is not None else '-',
                'Mínimo': f"{minimo:.2f}",
                'Q1 (25%)': f"{q1:.2f}",
                'Q3 (75%)': f"{q3:.2f}",
                'Máximo': f"{maximo:.2f}",
                'IQR': f"{iqr:.2f}",
                'Outliers': outliers,
                '% Outliers': f"{(outliers / total_registros * 100):.2f}%",
                'Assimetria': f"{assimetria:.2f}",
                'Curtose': f"{curtose:.2f}",
            })

        ### Agora faendo o processamento de cada categoria
        elif is_categorical:
            value_counts = col_data.value_counts()

            stats.update({
                'Zeros': '-',
                '% Zeros': '-',
                'Média': '-',
                'Mediana': '-',
                'Moda': value_counts.index[0] if len(value_counts) > 0 else '-',
                'Desvio Padrão': '-',
                'CV (%)': '-',
                'Mínimo': '-',
                'Q1 (25%)': '-',
                'Q3 (75%)': '-',
                'Máximo': '-',
                'IQR': '-',
                'Outliers': '-',
                '% Outliers': '-',
                'Assimetria': '-',
                'Curtose': '-',
                'Categorias': ', '.join(map(str, col_data.cat.categories.tolist())),
                'Mais Frequente': value_counts.index[0] if len(value_counts) > 0 else '-',
                'Freq. Mais Comum': value_counts.iloc[0] if len(value_counts) > 0 else '-',
                '% Mais Comum': f"{(value_counts.iloc[0] / total_registros * 100):.2f}%" if len(value_counts) > 0 else '-',
            })

        ### outros tipos
        else:
            moda_values = col_data.mode()

            stats.update({
                'Zeros': '-',
                '% Zeros': '-',
                'Média': '-',
                'Mediana': '-',
                'Moda': moda_values[0] if len(moda_values) > 0 else '-',
                'Desvio Padrão': '-',
                'CV (%)': '-',
                'Mínimo': str(col_data.min()) if len(col_data) > 0 else '-',
                'Q1 (25%)': '-',
                'Q3 (75%)': '-',
                'Máximo': str(col_data.max()) if len(col_data) > 0 else '-',
                'IQR': '-',
                'Outliers': '-',
                '% Outliers': '-',
                'Assimetria': '-',
                'Curtose': '-',
            })

        resumo[col] = pd.Series(stats)

    return pd.DataFrame(resumo).T


print("\n")
print("ESTATÍSTICAS DESCRITIVAS COMPLETAS")
print("\n")

estatisticas = calcular_estatisticas_detalhadas(df)
display(estatisticas)

### Salvar o resultado como csv
estatisticas.to_csv('estatisticas_descritivas.csv')
print("\nEstatísticas salvas como 'estatisticas_descritivas.csv'")

EDA e gráficos


In [ ]:
### EDA E GRÁFICOS

df_numeric = df.select_dtypes(include=[np.number])

### Observação: As colunas households, median_income, median_house_value e ocean_proximity não estão sendo identificadas como numéricos e precisam ser tratadas.

def limpar_e_converter_dados(df):
    df_limpo = df.copy()

    # Colunas que DEVEM ser numéricas
    colunas_numericas = ['longitude', 'latitude', 'housing_median_age','total_rooms', 'total_bedrooms', 'population','households', 'median_income', 'median_house_value']

    for col in colunas_numericas:
        if col in df_limpo.columns:
            ### Aqui eu converto para str primeiro
            df_limpo[col] = df_limpo[col].astype(str)

            ### Removendo espaços em branco
            df_limpo[col] = df_limpo[col].str.strip()

            ### Aqui eu removo as vírgulas
            df_limpo[col] = df_limpo[col].str.replace(',', '', regex=False)

            ### Removendo cifrões e outros símbolos monetários
            df_limpo[col] = df_limpo[col].str.replace('$', '', regex=False)
            df_limpo[col] = df_limpo[col].str.replace('R$', '', regex=False)

            ### Aqui eu removo os espaços extras que podem ter ficado
            df_limpo[col] = df_limpo[col].str.replace(' ', '', regex=False)

            ### Aqui eu converto para numérico (erros viram NaN)
            df_limpo[col] = pd.to_numeric(df_limpo[col], errors='coerce')

            print(f"{col}: convertido para {df_limpo[col].dtype}")

    ### ocean_proximity pode ser categorica
    if 'ocean_proximity' in df_limpo.columns:
        df_limpo['ocean_proximity'] = df_limpo['ocean_proximity'].astype(str).str.strip()
        df_limpo['ocean_proximity'] = df_limpo['ocean_proximity'].astype('category')
        print(f"ocean_proximity: convertido para categoria (não é numérico!)")

    return df_limpo

df_limpo = limpar_e_converter_dados(df)

### Verificar tipos
print("\n")
print("TIPOS DE DADOS APÓS ESSE TRATAMENTO:")
print("\n")
print(df_limpo.dtypes)
print("\n")

### Selecionar apenas colunas numéricas
df_numeric = df_limpo.select_dtypes(include=[np.number])
print("\n")
print(f"COLUNAS NUMÉRICAS IDENTIFICADAS: {len(df_numeric.columns)}")
print("\n")
print(df_numeric.columns.tolist())
print("\n")

### Calcular estatísticas novamente
estatisticas = calcular_estatisticas_detalhadas(df_limpo)
display(estatisticas)

In [ ]:
### GRÁFICO PARA MEDIDAS DE LOCALIZAÇÃO

### Decidi usar boxplots pois permitem comparação visual direta entre diferentes grupos e,
### por serem compactos, permitem visualizar múltiplas variáveis lado a lado para análise comparativa

def boxplots_por_localizacao(df_limpo):
    variaveis = ['median_house_value', 'median_income', 'housing_median_age', 'population']
    cores = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8']

    ### Extraindo um insight importante de cada gráfico:
    insights = {
        'median_house_value': """
INSIGHTS: Median House Value
- Bairros mais próximos do oceano (<1H OCEAN, NEAR OCEAN, NEAR BAY, ISLAND) têm valores medianos de imóveis claramente mais altos que INLAND.
- A dispersão é maior nas regiões costeiras, indicando maior heterogeneidade de preços e presença de imóveis muito caros.
""",
        'median_income': """
INSIGHTS: Median Income
- A renda mediana segue o padrão do valor dos imóveis: categorias mais próximas do oceano concentram rendas mais altas.
- INLAND apresenta mediana e média de renda menores do que <1H OCEAN e NEAR OCEAN, sugerindo que regiões costeiras concentram moradores com maior poder aquisitivo.
""",
        'housing_median_age': """
INSIGHTS: Housing Median Age
- As diferenças entre categorias de ocean_proximity são menos extremas do que em renda e valor do imóvel.
- Algumas áreas costeiras apresentam bairros um pouco mais antigos, o que pode indicar cidades mais consolidadas historicamente.
- Regiões INLAND parecem ter mistura maior de bairros novos e antigos, refletindo expansão urbana mais recente em algumas áreas.
""",
        'population': """
INSIGHTS: Population
- Há grande variabilidade de população em todas as categorias.
- Existem quarteirões muito densos tanto em regiões costeiras quanto em partes INLAND (grandes cidades interioranas).
- ISLAND tende a ter população mais baixa e menos dispersão, sugerindo poucos registros e bairros menos densos.
- No geral, a proximidade ao oceano não explica tanto a população quanto explica renda e valor do imóvel; a densidade populacional está mais espalhada entre as categorias.
"""
    }

    for var in variaveis:
        if var in df_limpo.columns and 'ocean_proximity' in df_limpo.columns:

            fig, ax = plt.subplots(figsize=(12, 7))

            ### Criando o bloxplot
            sns.boxplot(
                data=df_limpo,
                x='ocean_proximity',
                y=var,
                ax=ax,
                palette=cores,
                width=0.6,
                linewidth=2
            )

            medias = df_limpo.groupby('ocean_proximity')[var].mean()
            posicoes = range(len(medias))
            ax.scatter(
                posicoes, medias, color='red', s=100,
                zorder=10, label='Média', marker='D'
            )

            ### Criei uma linha que representa a mediana
            mediana_geral = df_limpo[var].median()
            ax.axhline(
                mediana_geral, color='darkred', linestyle='--', linewidth=2, alpha=0.5,
                label=f'Mediana Geral: {mediana_geral:,.0f}'
            )

            ### Aqui eu fiz a formatação
            ax.set_title(
                f'{var.replace("_", " ").title()}',
                fontsize=14, weight='bold', pad=15
            )
            ax.set_xlabel('Proximidade do Oceano', fontsize=11, weight='bold')
            ax.set_ylabel(var.replace("_", " ").title(), fontsize=11, weight='bold')
            ax.tick_params(axis='x', rotation=45)
            ax.grid(axis='y', alpha=0.3, linestyle='--')
            ax.legend(loc='upper right')

            plt.tight_layout(pad=3.0)
            fig.subplots_adjust(top=0.9, bottom=0.15)

            plt.savefig(f'boxplot_localizacao_{var}.png', dpi=300, bbox_inches='tight')
            print(f'Gráfico salvo como: boxplot_localizacao_{var}.png')
            plt.show()
            plt.close(fig)

            ### Printando os insights
            if var in insights:
                print("\n")
                print(insights[var].strip())
                print("\n" + "-"*80 + "\n")

boxplots_por_localizacao(df_limpo)

In [ ]:
### GRÁFICO PARA MEDIDAS DE DISPERSÃO

### Decidi usar um gráfico de barras simples para representar as medidas de dispersão já que o comprimento da barra
### pode medir a magnitude da dispersão de forma muito prática visualmente, não precisando de um conhecimento estatístico
### muito avançado.

def grafico_cv_por_grupo(df_limpo):

    ### Irei trabalhar com o coeficiente de variação. Escolhi porque ele permite comparar
    ### a dispersão relativa entre variáveis com escalas completamente diferentes.

    variaveis = ['median_house_value', 'median_income', 'housing_median_age', 'population']

    ### Comentários gerais pra cada gráfico
    insights_cv = {
        'median_house_value': """
INSIGHTS: Dispersão (Median House Value)
- O coeficiente de variação revela o quão heterogêneos são os preços dos imóveis dentro de cada grupo de proximidade ao oceano.
- Grupos com CV mais alto indicam mercados imobiliários mais desiguais, onde coexistem imóveis muito baratos e muito caros.
""",
        'median_income': """
INSIGHTS: Dispersão (Median Income)
- O CV da renda mostra o quanto a renda da população varia dentro de cada grupo.
- Grupos com CV mais alto sugerem maior desigualdade de renda entre os moradores daquela região.
""",
        'housing_median_age': """
INSIGHTS: Dispersão (Housing Median Age)
- O CV da idade mediana das casas indica se os bairros daquele grupo são mais homogêneos (casas de épocas parecidas)
  ou se há mistura de construções antigas e recentes.
""",
        'population': """
INSIGHTS: Dispersão (Population)
- O CV da população mostra quanta variação existe no tamanho dos quarteirões dentro de cada grupo de proximidade ao oceano.
- Grupos com CV mais alto reúnem desde quarteirões pouco povoados até áreas extremamente densas.
"""}

    for i, var in enumerate(variaveis):
        if var in df_limpo.columns and 'ocean_proximity' in df_limpo.columns:

            ### Criar figura individual para cada variável
            fig, ax = plt.subplots(figsize=(10, 6))

            ### Calculando o cv
            cv_por_grupo = (
                df_limpo
                .groupby('ocean_proximity')[var]
                .apply(lambda x: (x.std() / x.mean() * 100) if x.mean() != 0 else 0)
                .sort_values(ascending=False)
            )

            ### Organizando o gráfico de barras
            bars = ax.barh(
                range(len(cv_por_grupo)),
                cv_por_grupo.values,
                color='blue',
                alpha=0.8,
                edgecolor='black',
                linewidth=1.5
            )

            ### Adicionar valores nas barras
            for j, (bar, val) in enumerate(zip(bars, cv_por_grupo.values)):
                ax.text(
                    val + 1,
                    bar.get_y() + bar.get_height()/2,
                    f'{val:.1f}%',
                    va='center',
                    fontsize=10,
                    weight='bold'
                )

            ### Formatação dos eixos e título
            ax.set_yticks(range(len(cv_por_grupo)))
            ax.set_yticklabels(cv_por_grupo.index, fontsize=10)
            ax.set_xlabel('Coeficiente de Variação (%)', fontsize=11, weight='bold')
            ax.set_title(
                f'Dispersão Relativa - {var.replace("_", " ").title()}',
                fontsize=14,
                weight='bold',
                pad=15
            )
            ax.grid(axis='x', alpha=0.3, linestyle='--')

            ### CV médio
            cv_medio = cv_por_grupo.mean()
            ax.axvline(
                cv_medio,
                color='darkred',
                linestyle='--',
                linewidth=2,
                alpha=0.7,
                label=f'CV Médio: {cv_medio:.1f}%'
            )
            ax.legend()

            plt.tight_layout()
            plt.savefig(f'cv_dispersao_{var}.png', dpi=300, bbox_inches='tight')

            ### Salvando os gráficos:
            print(f'Gráfico salvo: cv_dispersao_{var}.png')
            plt.show()
            plt.close(fig)

            ### Comentário (insight) após cada gráfico
            cat_max = cv_por_grupo.idxmax()
            val_max = cv_por_grupo.max()
            cat_min = cv_por_grupo.idxmin()
            val_min = cv_por_grupo.min()

            print("\n")
            if var in insights_cv:
                print(insights_cv[var].strip())
            print(f"Maior CV em: {cat_max} ({val_max:.1f}%)")
            print(f"Menor CV em: {cat_min} ({val_min:.1f}%)")
            print("\n"+"-"*60+"\n")


grafico_cv_por_grupo(df_limpo)

In [ ]:
### GRÁFICO PARA VALORES ÚNICOS (CARDINALIDADE)

### Escolhi gráfico de barras horizontal porque facilita a leitura dos nomes das colunas (sem rotação),
### permite comparação imediata através do comprimento das barras e usa cores para classificar automaticamente
### as variáveis (categóricas em vermelho, média em laranja, alta cardinalidade em azul). Essa visualização
### é essencial para identificar a diversidade dos dados.

from matplotlib.patches import Patch

def grafico_valores_unicos_horizontal(df_limpo):
    valores_unicos = df_limpo.nunique().sort_values(ascending=True)
    total_registros = len(df_limpo)

    fig, ax = plt.subplots(figsize=(10, 8))

    ### Escolhendo as cores.
    cores = []
    for v in valores_unicos.values:
        if v < 10:
            cores.append('#E74C3C')  ### Vermelho - Categórica
        elif v < 1000:
            cores.append('#F39C12')  ### Laranja - Média
        else:
            cores.append('#3498DB')  ### Azul - Alta

    bars = ax.barh(
        range(len(valores_unicos)),
        valores_unicos.values,
        color=cores,
        alpha=0.8,
        edgecolor='black',
        linewidth=1.5
    )

    for i, (bar, val) in enumerate(zip(bars, valores_unicos.values)):
        width = bar.get_width()
        ax.text(
            width + total_registros*0.02,
            bar.get_y() + bar.get_height()/2,
            f'{val:,}',
            va='center',
            fontsize=10,
            weight='bold'
        )

    ### Formatação.
    ax.set_yticks(range(len(valores_unicos)))
    ax.set_yticklabels(valores_unicos.index, fontsize=11)
    ax.set_xlabel('Quantidade de Valores Únicos', fontsize=12, weight='bold')
    ax.set_title(
        'Cardinalidade das Variáveis (Valores Únicos)',
        fontsize=14,
        weight='bold',
        pad=15
    )
    ax.grid(axis='x', alpha=0.3, linestyle='--')

    ### Linha de referência: total de registros
    ax.axvline(
        total_registros,
        color='darkred',
        linestyle='--',
        linewidth=2,
        alpha=0.5,
        label=f'Total: {total_registros:,}'
    )

    ### Legenda do gráfico:
    legend_elements = [
        Patch(facecolor='#E74C3C', label='Categórica (< 10)'),
        Patch(facecolor='#F39C12', label='Média (10-1000)'),
        Patch(facecolor='#3498DB', label='Alta (> 1000)'),
        Patch(facecolor='none', edgecolor='darkred', linestyle='--', label='Total de Registros')
    ]
    ax.legend(handles=legend_elements, loc='lower right')

    plt.tight_layout()
    plt.savefig('cardinalidade_horizontal.png', dpi=300, bbox_inches='tight')

    ### Salvando o gráfico
    print('Gráfico salvo: cardinalidade_horizontal.png')
    plt.show()
    plt.close(fig)

    # ---------------- INSIGHTS APÓS O GRÁFICO ----------------
    col_max = valores_unicos.idxmax()
    val_max = valores_unicos.max()
    col_min = valores_unicos.idxmin()
    val_min = valores_unicos.min()

    qtd_baixa = (valores_unicos < 10).sum()
    qtd_media = ((valores_unicos >= 10) & (valores_unicos < 1000)).sum()
    qtd_alta = (valores_unicos >= 1000).sum()

    print("""
INSIGHTS:
- O gráfico mostra como cada coluna varia em termos de quantidade de valores distintos (diversidade de informação).
- Variáveis com baixa cardinalidade (< 10) tendem a ser boas candidatas a variáveis categóricas.
- Variáveis com cardinalidade muito alta podem exigir tratamentos específicos (por exemplo, agrupar categorias raras).
""".strip())
    print(f"\nColuna com MAIOR cardinalidade: {col_max} ({val_max:,} valores únicos)")
    print(f"Coluna com MENOR cardinalidade: {col_min} ({val_min:,} valores únicos)")
    print(f"\nQuantidade de variáveis com baixa cardinalidade (< 10): {qtd_baixa}")
    print(f"Quantidade de variáveis com cardinalidade média (10–1000): {qtd_media}")
    print(f"Quantidade de variáveis com alta cardinalidade (> 1000): {qtd_alta}")
    print("\n"+"-"*60+"\n")

grafico_valores_unicos_horizontal(df_limpo)

In [ ]:
### GRÁFICO GEOGRÁFICO: Mapa de Preços e Densidade Populacional
### Este gráfico é essencial porque explora a dimensão espacial dos dados, revelando
### padrões geográficos que nao aparecem em estatísticas agregadas. O tamanho dos pontos
### representa população e a cor representa preço, permitindo identificar visualmente
### áreas metropolitanas caras (São Francisco, Los Angeles) e regiões mais baratas (interior).

def mapa_geografico_precos(df_limpo):

    ### MAPA 1: PREÇO MEDIANO POR LOCALIZAÇÃO

    fig1, ax1 = plt.subplots(figsize=(12, 10))

    scatter1 = ax1.scatter(
        df_limpo['longitude'],
        df_limpo['latitude'],
        c=df_limpo['median_house_value'],
        s=df_limpo['population'] / 100,
        cmap='YlOrRd',
        alpha=0.4,
        edgecolors='k',
        linewidth=0.1
    )

    cbar1 = plt.colorbar(scatter1, ax=ax1)
    cbar1.set_label('Valor Mediano da Casa', fontsize=11, weight='bold')

    ### Formatação
    ax1.set_xlabel('Longitude', fontsize=12, weight='bold')
    ax1.set_ylabel('Latitude', fontsize=12, weight='bold')
    ax1.set_title('Distribuição Geográfica dos Preços de Imóveis\n(Tamanho = População)',
                  fontsize=14, weight='bold', pad=15)
    ax1.grid(alpha=0.3)

    plt.tight_layout()
    plt.savefig('mapa_precos_california.png', dpi=300, bbox_inches='tight')
    print('Gráfico salvo como: mapa_precos_california.png')
    plt.show()
    plt.close(fig1)

    ### INSIGHTS DO MAPA DE PREÇOS
    print("""
INSIGHTS:
- A análise espacial revela que a localização é o fator dominante no preço dos imóveis da Califórnia.
- Observa-se um gradiente claro de valorização do litoral para o interior: regiões costeiras
  apresentam, em geral, casas mais caras que o interior.
- Três grandes clusters metropolitanos (Bay Area, Los Angeles e San Diego) concentram os maiores preços,
  reforçando o papel de grandes centros urbanos e da proximidade ao oceano na valorização imobiliária.
- Nota-se também heterogeneidade local: mesmo dentro de uma mesma região, distritos vizinhos podem
  apresentar variações de cerca de 50% no preço, mostrando que microlocalizações importam muito.
""".strip())
    print("\n"+"-"*60 +"\n")


    ### MAPA 2: RENDA MEDIANA POR LOCALIZAÇÃO

    fig2, ax2 = plt.subplots(figsize=(12, 10))

    scatter2 = ax2.scatter(
        df_limpo['longitude'],
        df_limpo['latitude'],
        c=df_limpo['median_income'],
        s=df_limpo['population'] / 100,
        cmap='viridis',
        alpha=0.4,
        edgecolors='k',
        linewidth=0.1
    )

    cbar2 = plt.colorbar(scatter2, ax=ax2)
    cbar2.set_label('Renda Mediana (×$10,000)', fontsize=11, weight='bold')

    ### Formatação
    ax2.set_xlabel('Longitude', fontsize=12, weight='bold')
    ax2.set_ylabel('Latitude', fontsize=12, weight='bold')
    ax2.set_title('Distribuição Geográfica da Renda\n(Tamanho = População)',
                  fontsize=14, weight='bold', pad=15)
    ax2.grid(alpha=0.3)

    plt.tight_layout()
    plt.savefig('mapa_renda_california.png', dpi=300, bbox_inches='tight')
    print('Gráfico salvo como: mapa_renda_california.png')
    plt.show()
    plt.close(fig2)

    ### INSIGHTS DO MAPA DE RENDA
    print("""
INSIGHTS:
- Os clusters de maior renda coincidem fortemente com as mesmas regiões de preços elevados
  (Bay Area, Los Angeles e San Diego), indicando que renda e preço têm correlação não só numérica,
  mas também geográfica.
- A proximidade do oceano é crítica: áreas costeiras concentram não apenas imóveis mais caros,
  mas também moradores com maior poder aquisitivo.
- Comparando os dois mapas (preço e renda), fica evidente que casas costeiras podem valer de 2 a 3 vezes
  mais do que imóveis semelhantes no interior, reforçando o papel central da localização na dinâmica do mercado.
""".strip())
    print("\n"+"-"*60+"\n")

mapa_geografico_precos(df_limpo)

Feature Engineering e Dados Categóricos

In [ ]:
df_fe = df_limpo.copy()

### A combinação de variáveis de “totais” em razões/densidades é importante porque os
### totais brutos (total_rooms, total_bedrooms, population, households) são altamente correlacionados
### entre si e refletem mais o tamanho do distrito do que suas características estruturais.
### Ao transformá‑los em métricas relativas, como rooms_per_household, population_per_household e bedrooms_per_room, passamos a medir intensidade
### e composição (tamanho médio das casas, lotação média por domicílio, proporção de quartos), que são muito mais comparáveis entre regiões de
### tamanhos diferentes.

num_cols_needed = ['total_rooms', 'total_bedrooms', 'population', 'households']
if all(col in df_fe.columns for col in num_cols_needed):
    ### Evitando a divisão por 0
    for col in ['households', 'total_rooms']:
        df_fe.loc[df_fe[col] == 0, col] = np.nan

    ### Features novas criadas:
    df_fe['rooms_per_household'] = df_fe['total_rooms'] / df_fe['households']
    df_fe['population_per_household'] = df_fe['population'] / df_fe['households']
    df_fe['bedrooms_per_room'] = df_fe['total_bedrooms'] / df_fe['total_rooms']

    ### Removendo as colunas originais
    cols_to_drop = ['total_rooms', 'total_bedrooms', 'population', 'households']
    df_fe.drop(columns=[c for c in cols_to_drop if c in df_fe.columns], inplace=True)

### Transformar latitude e longitude em um “cluster de região” (por exemplo, via K‑Means) serve para capturar a informação geográfica de forma mais simples e útil ao modelo.
### Em vez de lidar com duas coordenadas contínuas e muito granulares, o modelo passa a enxergar poucas regiões discretas que representam áreas com
### comportamento imobiliário semelhante (ex.: grandes regiões metropolitanas, interior, litoral).
### Isso reduz a dimensionalidade, diminui o ruído de microdiferenças de localização (quarteirão a quarteirão) e facilita a generalização:
### o foco deixa de ser a coordenada exata e passa a ser a macro‑região econômica em que o imóvel está inserido, que é o que realmente influencia preço,
### renda e outras variáveis de interesse.

if {'latitude', 'longitude'}.issubset(df_fe.columns):
    coords = df_fe[['latitude', 'longitude']]

    ### Aqui eu uso KMeans para agrupar a Califórnia em K regiões
    kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
    df_fe['region_cluster'] = kmeans.fit_predict(coords)

    ### Removendo as colunas originais
    df_fe.drop(columns=['latitude', 'longitude'], inplace=True)


df_fe

In [ ]:
### Uma forma correta de fazer isso é usar codificação one‑hot, criando colunas binárias 0/1 para indicar a presença de cada categoria.
### No caso da questão, primeiro simplifiquei a variável ocean_proximity em dois grupos (INLAND e COASTAL) e, em seguida, apliquei pd.get_dummies para gerar
### uma coluna numérica (ocean_COASTAL) que vale 1 para distritos costeiros e 0 para os do interior.
### Assim, removemos a coluna categórica original (ocean_proximity) e ficamos apenas com atributos numéricos,
### permitindo que o modelo interprete corretamente esse fator geográfico sem assumir nenhuma ordem artificial entre categorias.

if 'ocean_proximity' in df_fe.columns:
    ### Tudo que não é INLAND vira COASTAL
    df_fe['ocean_proximity_grp'] = np.where(
        df_fe['ocean_proximity'] == 'INLAND',
        'INLAND',
        'COASTAL'
    )

    ### Removendo a coluna original
    df_fe.drop(columns=['ocean_proximity'], inplace=True)

    ### Aplicando One-hot encoding da categoria reduzida (INLAND x COASTAL)
    dummies_ocean = pd.get_dummies(
        df_fe['ocean_proximity_grp'],
        prefix='ocean',
        drop_first=True
    )

    df_fe = pd.concat(
        [df_fe.drop(columns=['ocean_proximity_grp']), dummies_ocean],
        axis=1
    )

df_fe